 EPIC, the Extended PID Configurator, Ihr vertrauenswürdiger Konfigurator :)
V0.1 := Initial release.

#Writed according PEP8 style guide
#https://realpython.com/python-pep8/

In [2]:
#Libraries import
import os
import time
import datetime
import pandas as pd
import numpy as np
from numpy import random
from enum import Enum
#import scipy.sparse as sp
#from sklearn.datasets import load_iris


##Version information
print(f"Pandas version: {pd.__version__}") #Pandas version number
print(f"Numpy version: {np.__version__}") #numpy version number

pd.options.display.float_format = '{:.3f}'.format



Pandas version: 1.4.3
Numpy version: 1.23.1


In [3]:
#Global variables
import_mode = True # import mode (True/False) - Imports an csv from PID
verbose_mode = False # verbose mode (True/False)
debug_mode = True # debug mode (True/False)
random_mode = False # random data generation  (True/False)
simulation_mode = False # simulation (True/False)

In [4]:
#Generate standard templated
generate_tag_number_definitions = False #Every element has a description and a unique number for software modules
#generate_element_list_alarm_matrix = False #Every element has a unique number of alarms
#generate_element_list_hardware_matrix = False #All possible combinations of elements have a unique IO

In [5]:
#Data generation
generate_element_list = False
generate_functional_matrix = False
generate_alarm_matrix = False
generate_hardware_config = False
generate_symbolic   = False
generate_s_plant    = False
generate_s_line = False
generate_flow_chart = False #--> GUI with active modes and phases, start condition [active modes list ] as bool (true false). Generarte excel which generete flow chart.


In [6]:
if import_mode: 
    ##Change working directory to temp/epic
    os.chdir('C://temp/epic/')
    cwd = os.getcwd()
    print('Current Working Directory is: ', cwd)
    


Current Working Directory is:  C:\temp\epic


In [7]:
if import_mode:
    print(os.listdir(cwd)) #List directory

['AM.xlsm', 'AutomationCode.xlsx', 'Element_list_export.xlsx', 'Element_list_export_padagis.xlsx', 'GroupElementList.xlsx', 'Info.txt', 'OLD_PID.csv', 'PID - Piping and Instrument Diagram_394451_A03_NINIO.vsdm', 'PID - Piping and Instrument Diagram_400173_B02_SVEN.vsdm', 'PID - Piping and Instrument Diagram_406699_A01 - DOSING_UNITS.vsdm', 'PID - Piping and Instrument Diagram_406699_A01.vsdm', 'PID.xlsx', 'PID_NINIO.xlsx', 'PID_SVEN.xlsx', 'TND.xlsx', '~$$PID - Piping and Instrument Diagram_406699_A01 - DOSING_UNITS.~vsdm', '~$$PID - Piping and Instrument Diagram_406699_A01.~vsdm', '~$AutomationCode.xlsx', '~$Element_list_export_padagis.xlsx']


#Random_mode:
Generate aleatories PID numbers and symbols.



In [8]:
#Generate list of X random numbers from 1.100 - 9.999:
if random_mode:
    random_PID_numbers=list(random.randint(1100,10000, size=(1000)))

    if verbose_mode:    print(random_PID_numbers)

In [9]:
#Generate PID symbol
if random_mode:
    random_PID_symbols = list(random.choice(['B', 'C', 'D', 'EC', 'F', 'FA-', 'FIA+', 'FICA+', 'FSA-', 'GIA', 'GSA+', 'GSA-', 'GZA-', 'H', 'K', 'L', 'LSA-', 'M',
    'MIA±', 'NC', 'P', 'PDI', 'PDICA±', 'PDIS', 'PI', 'PSA-', 'QIA+', 'QIA-', 'QSA+', 'S', 'SC', 'SIA±', 'T', 'TI', 'TIC', 'TICA±', 'TSA+',
    'V', 'W', 'WICA±', 'XA', 'Y', 'YC'], size=(1000)))

    if verbose_mode:    print(random_PID_symbols)
    if debug_mode:      print(type(random_PID_symbols))

Series and DataFrames

In [10]:
#pid_series = pd.Series(random_PID_symbols, random_PID_numbers)
#pid_series = pid_series.sort_index(axis=0,ascending=True)
#pid_series

#Import Tag number definitions as a pandas series
#tnd = pd.read_csv('TND.csv',names=['PID','Description'])
#tnd = pd.read_excel('TND.xlsx',)
#tnd

In [11]:
#PID_series = tnd['PID'].squeeze()
#pid_exist = PID_series.str.contains('.100')

In [12]:
#tnd.at[0,'Description']

In [13]:
#tnd_pid.at[1.100,'Description'] Works if index_col='PID'

Reading Excel File

We can read an excel file into a data frame:

pd.read_excel('myExcel.xlsx', index_col=['ColumnA'])

We can also load one sheet into a data frame:

pd.read_excel(open('myExcel.xlsx', 'rb'), sheet_name='Sheet1')

AUTOMATION CODE

In [14]:
#Load PID data
#df_data_excel = pd.read_excel('PID.xlsx', index_col='Prop_PID_Number')
df_data_excel = pd.read_excel('PID.xlsx')
#df_data_excel = pd.read_excel('PID_SVEN.xlsx')
#df_data_excel = pd.read_excel('PID_NINIO.xlsx')
df_data_excel = df_data_excel.sort_values('Prop_PID_Element') #Sorting by Element we get elements ordered and ready to compare
print(df_data_excel)

     ShapeKey Prop_PID_Element  Prop_PID_Number
21       1762                B            1.852
179     14496                C            1.155
175     14444                D            1.680
177     16120               EC            1.680
0       18271                F            1.221
..        ...              ...              ...
45      16176               YC            1.717
7       16200               YC            1.555
105     17727               YC            2.717
279     16197               YC            1.560
126     18416               YC            1.727

[307 rows x 3 columns]


In [15]:
#Remove ShapeKey from the dataframe
del df_data_excel['ShapeKey']

In [16]:
# dropping duplicate values
df_data_excel.drop_duplicates(inplace=True)

In [17]:
df_data_excel.count()

Prop_PID_Element    299
Prop_PID_Number     299
dtype: int64

In [18]:
#Drop elements like filters, catalizators, etc
for items in df_data_excel.index:

    if df_data_excel.at[items, 'Prop_PID_Element'] == 'B':
        print("Deleted a catalizator in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'F':
        print("Deleted a filter in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'C':
        print("Deleted a catalizator in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)

    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'D':
        print("Deleted a D element: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    
    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'S':
        print("Deleted a Overpressure valve in: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)

    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'W':
        print("Deleted a W element: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)

    elif df_data_excel.at[items, 'Prop_PID_Element'] == 'Z':
        print("Deleted a Z element: ",items)
        df_data_excel.drop(items,axis=0,inplace=True)
    




Deleted a catalizator in:  21
Deleted a catalizator in:  179
Deleted a D element:  175
Deleted a filter in:  0
Deleted a filter in:  77
Deleted a filter in:  40
Deleted a filter in:  24
Deleted a filter in:  23
Deleted a filter in:  19
Deleted a filter in:  123
Deleted a filter in:  255
Deleted a filter in:  256
Deleted a filter in:  189
Deleted a filter in:  119
Deleted a filter in:  107
Deleted a filter in:  76
Deleted a filter in:  302
Deleted a filter in:  75
Deleted a filter in:  74
Deleted a filter in:  72
Deleted a filter in:  285
Deleted a filter in:  118
Deleted a filter in:  284
Deleted a filter in:  84
Deleted a filter in:  90
Deleted a filter in:  271
Deleted a filter in:  66
Deleted a filter in:  102
Deleted a filter in:  190
Deleted a Overpressure valve in:  50
Deleted a Overpressure valve in:  9
Deleted a W element:  270


In [19]:
df_data_excel.count()


Prop_PID_Element    267
Prop_PID_Number     267
dtype: int64

In [20]:
#Load Automation Code excel file
df_automation_code = pd.read_excel('AutomationCode.xlsx')
df_automation_code.head()

,Prop_PID_Element,Automation_Code
0,B,* Buffer *
1,C,* Catalizator *
2,D,* D *
3,EC,Voltage Control
4,F,* Filter *


In [21]:
#Load Tag Name Definition
tnd_pid = pd.read_excel('TND.xlsx')
#tnd_pid = pd.read_excel('TND.xlsx',index_col='PID')
tnd_pid

,PID,Description,Unnamed: 2,S_LIMIT_HI_LO[1],S_LIMIT_HI_LO[2],S_LIMIT_HI_LO[3],S_LIMIT_HI_LO[4],S_LIMIT_SWITCH,S_FLOW_METER,LIMIT_ON_OFF,A_AIRSAPLETr_DIGITAL
0,Y.100,Intake air fan,Intake air fan (including service switch),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Y.101,Filter 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Y.102,Filter 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Y.103,Filter 3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Y.104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
895,Y.995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
896,Y.996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
897,Y.997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
898,Y.998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Def of Lookup as in Microsoft Excel.
def xlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [23]:
# Lookup for PID number: 1.840 or 2.840 return the same definition: Diff. pressure sensor
def xlookupPID(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    lookup_value = str("%.3f" % lookup_value)
    lookup_value = 'Y' + lookup_value[1:]
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return f'"{lookup_value}" not found!' if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [24]:
df_data_excel.count()

Prop_PID_Element    267
Prop_PID_Number     267
dtype: int64

In [25]:
print(df_data_excel.to_string())

    Prop_PID_Element  Prop_PID_Number
177               EC            1.680
146              FA-            1.300
156              FA-            1.330
162              FA-            2.300
249              FA-            1.320
209              FA-            1.310
42              FIA+            1.825
109            FICA±            2.728
47             FICA±            1.708
104            FICA±            2.718
44             FICA±            1.718
125            FICA±            1.728
304            FICA±            2.708
213             FSA-            1.338
120             FSA-            2.308
112             FSA-            1.328
216             FSA-            1.318
96              FSA-            1.308
280              GIA            1.560
8                GIA            1.555
12               GIA            1.550
300             GIA+            1.400
266             GIA+            1.410
301             GIA-            1.410
261             GIA-            1.400
199         

Add extra elements like 1.840 [Decontamionation], later will count as a Limit Hi Lo, for the alarms

In [26]:
#Add Automation_Code information
#df_data_excel['Automation_Code'] = df_data_excel['Prop_PID_Element'].apply(xlookup, args = (df_automation_code['Prop_PID_Element'], df_automation_code['Automation_Code']));

In [27]:
#print(df_data_excel.to_string())


In [28]:
#Group by PID number
element_list = df_data_excel.groupby(['Prop_PID_Number']).agg(list).copy()

In [29]:
type(element_list)

pandas.core.frame.DataFrame

In [30]:
element_list.count()

Prop_PID_Element    144
dtype: int64

In [31]:
element_list.describe()

,Prop_PID_Element
count,144
unique,36
top,[GZA-]
freq,21


In [32]:
print(element_list.to_string())

                        Prop_PID_Element
Prop_PID_Number                         
1.100                     [M, SC, V, XA]
1.108                              [PDI]
1.150                   [M, SC, TSA+, V]
1.158                              [PDI]
1.160                     [M, SC, V, XA]
1.200                     [M, SC, V, XA]
1.208                              [PDI]
1.210                     [M, SC, V, XA]
1.218                              [PDI]
1.240                     [M, SC, V, XA]
1.248                              [PDI]
1.300                    [FA-, QIA+, XA]
1.304                             [M, P]
1.305                 [GSA+, GSA-, V, Y]
1.308                             [FSA-]
1.310                    [FA-, QIA+, XA]
1.314                             [M, P]
1.315                 [GSA+, GSA-, V, Y]
1.318                             [FSA-]
1.320                    [FA-, QIA+, XA]
1.324                             [M, P]
1.325                 [GSA+, GSA-, V, Y]
1.328           

In [33]:
#Drop compound elements which are not part of the software list
#Drop elements like filters, catalizators, etc
for items in element_list.index:
    if element_list.at[items, 'Prop_PID_Element'] == ['PI', 'V']:
        print("Deleted a Pressure indicator in: ",items)
        element_list.drop(items,axis=0,inplace=True)

Deleted a Pressure indicator in:  1.855


In [34]:
element_list.count()

Prop_PID_Element    143
dtype: int64

In [35]:
#Deletion of manual valves, as we have no other information in the PID
#element_list.drop(1.654,axis=0,inplace=True)
#element_list.drop(1.656,axis=0,inplace=True)
#element_list.drop(1.850,axis=0,inplace=True)
#element_list.drop(1.863,axis=0,inplace=True)

In [36]:
element_list.count()

Prop_PID_Element    143
dtype: int64

In [37]:
##Add tag number definition - reset index first, then use xlookup function
element_list = element_list.reset_index()
element_list['Tag_Number_Definition'] = element_list['Prop_PID_Number'].apply(xlookupPID, args = (tnd_pid['PID'], tnd_pid['Description']));

In [38]:
print(element_list.to_string())

     Prop_PID_Number         Prop_PID_Element                 Tag_Number_Definition
0              1.100           [M, SC, V, XA]                        Intake air fan
1              1.108                    [PDI]           Diferential pressure sensor
2              1.150         [M, SC, TSA+, V]                       Exhaust air fan
3              1.158                    [PDI]           Diferential pressure sensor
4              1.160           [M, SC, V, XA]                       Exhaust air fan
5              1.200           [M, SC, V, XA]                     Recirculation fan
6              1.208                    [PDI]          Diferential pressure sensor 
7              1.210           [M, SC, V, XA]                     Recirculation fan
8              1.218                    [PDI]           Diferential pressure sensor
9              1.240           [M, SC, V, XA]                     Recirculation fan
10             1.248                    [PDI]           Diferential pressure

Add elements that has two software main modules, as 1.825 Leak valve (motor,reg.limit) and 1.825 Leak (leak, limit)
SIS module and gasket are now possible to configure

index+1     1.825   [Leak] [Limit_hi_lo] 

In [39]:
#which returns True or False
if (1.655 and 1.680) in element_list.Prop_PID_Number.values: 
    print("Say hello")

Say hello


In [40]:
#Add data at the end of the dataframe
#add new row to end of DataFrame
#new_row = pd.DataFrame([1.825, '[LEAK]', 'S_LEAK main component'])
#element_list = element_list.append(pd.Series(new_row,index=element_list.columns[:len(new_row)]), ignore_index=True) #Deprecated, use concat instead
#element_list.loc[len(element_list.index)] = new_row

if (1.655 and 1.680) in element_list.Prop_PID_Number.values: 
    dicc_AHU01 = {'Prop_PID_Number': 'AHU01', 'Prop_PID_Element': 'AHU01' , 'Tag_Number_Definition': 'Leader/Follower AHU 01'}
    element_list = element_list.append(dicc_AHU01, ignore_index=True)



dicc_1800 = {'Prop_PID_Number': '1.800_precon', 'Prop_PID_Element': '[702] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
element_list = element_list.append(dicc_1800, ignore_index=True)

dicc_1800 = {'Prop_PID_Number': '1.800_deco', 'Prop_PID_Element': '[118] [17°C < 28°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
element_list = element_list.append(dicc_1800, ignore_index=True)

dicc_1800 = {'Prop_PID_Number': '1.800_pro', 'Prop_PID_Element': '[114] [15°C < 21°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
element_list = element_list.append(dicc_1800, ignore_index=True)

dicc_1800 = {'Prop_PID_Number': '1.800_aer1', 'Prop_PID_Element': '[708] [30°C < 40°C]' , 'Tag_Number_Definition': 'Temperature sensor'}
element_list = element_list.append(dicc_1800, ignore_index=True)


dicc_1810 = {'Prop_PID_Number': '1.810_pro', 'Prop_PID_Element': '[114] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
element_list = element_list.append(dicc_1810, ignore_index=True)

dicc_1810 = {'Prop_PID_Number': '1.810_precon', 'Prop_PID_Element': '[702] [15%RH < 20%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
element_list = element_list.append(dicc_1810, ignore_index=True)

dicc_1810 = {'Prop_PID_Number': '1.810_deco', 'Prop_PID_Element': '[118] [< 100%RH]' , 'Tag_Number_Definition': 'Humidity sensor'}
element_list = element_list.append(dicc_1810, ignore_index=True)



dicc_1820 = {'Prop_PID_Number': '1.820_pro', 'Prop_PID_Element': '[114] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
element_list = element_list.append(dicc_1820, ignore_index=True)

dicc_1820 = {'Prop_PID_Number': '1.820_deco', 'Prop_PID_Element': '[118] [> 0.20m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
element_list = element_list.append(dicc_1820, ignore_index=True)

dicc_1820 = {'Prop_PID_Number': '1.820_aer2', 'Prop_PID_Element': '[709] [> 0.45m/s]' , 'Tag_Number_Definition': 'Air velocity sensor'}
element_list = element_list.append(dicc_1820, ignore_index=True)


dicc_1825 = {'Prop_PID_Number': '1.825_leak', 'Prop_PID_Element': 'LEAK' , 'Tag_Number_Definition': 'Flow rate sensor'}
element_list = element_list.append(dicc_1825, ignore_index=True)


dicc_1840 = {'Prop_PID_Number': '1.840_prod', 'Prop_PID_Element': '[114] [>30Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
element_list = element_list.append(dicc_1840, ignore_index=True)

dicc_1840 = {'Prop_PID_Number': '1.840_deco', 'Prop_PID_Element': '[118] [>15Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
element_list = element_list.append(dicc_1840, ignore_index=True)

dicc_1840 = {'Prop_PID_Number': '1.840_leak', 'Prop_PID_Element': '[108] [60Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
element_list = element_list.append(dicc_1840, ignore_index=True)

dicc_1840 = {'Prop_PID_Number': '1.840_emy', 'Prop_PID_Element': '[105] [<-200Pa >200Pa]' , 'Tag_Number_Definition': 'Differencial pressure sensor'}
element_list = element_list.append(dicc_1840, ignore_index=True)

##Analog error goes also here
#Like 1.550 analog fault, 1.555





C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\4079352263.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  element_list = element_list.append(dicc_AHU01, ignore_index=True)
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\4079352263.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  element_list = element_list.append(dicc_1800, ignore_index=True)
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\4079352263.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  element_list = element_list.append(dicc_1800, ignore_index=True)
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\4079352263.py:20: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.con

In [41]:
print(element_list.to_string())

    Prop_PID_Number         Prop_PID_Element                 Tag_Number_Definition
0             1.100           [M, SC, V, XA]                        Intake air fan
1             1.108                    [PDI]           Diferential pressure sensor
2             1.150         [M, SC, TSA+, V]                       Exhaust air fan
3             1.158                    [PDI]           Diferential pressure sensor
4             1.160           [M, SC, V, XA]                       Exhaust air fan
5             1.200           [M, SC, V, XA]                     Recirculation fan
6             1.208                    [PDI]          Diferential pressure sensor 
7             1.210           [M, SC, V, XA]                     Recirculation fan
8             1.218                    [PDI]           Diferential pressure sensor
9             1.240           [M, SC, V, XA]                     Recirculation fan
10            1.248                    [PDI]           Diferential pressure sensor
11  

In [42]:
#print(element_list.loc[element_list['Prop_PID_Number'] == 1.100]['Prop_PID_Element'].values)

In [43]:
value =list( element_list.at[0, 'Prop_PID_Element'])

In [44]:
value

['M', 'SC', 'V', 'XA']

In [45]:
#Software Module Definition
SOFTWARE_MODE_DEFINITION = {
    "S_AAS_MAS_TCP" : "S_AAS_MAS_TCP",
    "S_AAS_MODBUS_TCP" : "S_AAS_MODBUS_TCP",
    "S_AAS_SERIAL" : "S_AAS_SERIAL",
    "S_ALARM" : "S_ALARM",
    "S_DRIVE" : "S_DRIVE",
    "S_ELMT_ON_OFF" : "S_ELMT_ON_OFF",
    "S_EMITTER_CMT" : "S_EMITTER_CMT",
    "S_FIPA" : "S_FIPA",
    "S_FLOW_METER" : "S_FLOW_METER",
    "S_GASKET" : "S_GASKET",
    "S_LEAK" : "S_LEAK",
    "S_LIMIT_HI_LO" : "S_LIMIT_HI_LO",
    "S_LIMIT_ON_OFF" : "S_LIMIT_ON_OFF",
    "S_LIMIT_SWITCH" : "S_LIMIT_SWITCH",
    "S_LINE" : "S_LINE",
    "S_MOTOR1" : "S_MOTOR1",
    "S_PLANT" : "S_PLANT",
    "S_PMS_MODBUS_ASCII" : "S_PMS_MODBUS_ASCII",
    "S_PMS_MODBUS_TCP" : "S_PMS_MODBUS_TCP",
    "S_PUMP_TEST" : "S_PUMP_TEST",
    "S_REG_CONT" : "S_REG_CONT",
    #S_REG = "S_REG"
    "S_SIS" : "S_SIS",
    "S_SKANFOG" : "SKANFOG",
    #S_TIMESYNCH = "S_TIMESYNCH"
    "S_VALVE_FLAP" : "S_VALVE_FLAP",
    #S_WEIGHT_SCALE_NG = "S_WEIGHT_SCALE_NG"
    #S_WEIGHT_SCALE = "S_WEIGHT_SCALE"
    "S_WEIGHT_SCALE2_NG" : "S_WEIGHT_SCALE2_NG",
    "S_LEADER_FOLLOWER" : "S_LEADER_FOLLOWER"
    }

In [46]:
element_list_sw_modules = ['SW_Module','SW_Module_Nr','Connection1','Connection1_Nr','Connection2','Connection2_Nr', 'Connection3','Connection3_Nr','Connection4','Connection4_Nr']
element_list[element_list_sw_modules] = 'NaN'
element_list

,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,1.100,"[M, SC, V, XA]",Intake air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.108,[PDI],Diferential pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.150,"[M, SC, TSA+, V]",Exhaust air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.158,[PDI],Diferential pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.160,"[M, SC, V, XA]",Exhaust air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,1.825_leak,LEAK,Flow rate sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,1.840_prod,[114] [>30Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,1.840_deco,[118] [>15Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,1.840_leak,[108] [60Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
#Set Pid numbers as column
#create a new dataframe with PID as index
#element_list = element_list.set_index('Prop_PID_Number')


In [48]:
#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
#response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
#print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
#print(index_of[0]) #[0] first element, [-1] last element

#element_list.SW_Module_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
#element_list.Prop_PID_Number[indice] = 'Change pid number'
#element_list.Connection3_Nr[indice] = 11
#using index
#element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

element_list

,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,1.100,"[M, SC, V, XA]",Intake air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.108,[PDI],Diferential pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.150,"[M, SC, TSA+, V]",Exhaust air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1.158,[PDI],Diferential pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.160,"[M, SC, V, XA]",Exhaust air fan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,1.825_leak,LEAK,Flow rate sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,1.840_prod,[114] [>30Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,1.840_deco,[118] [>15Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,1.840_leak,[108] [60Pa],Differencial pressure sensor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
drive_number = 0
motor_number = 0
reg_number = 0
limit_hi_lo = 0
elmt_on_off = 0
valve_flap = 0
limit_switch = 0
sis = 0
sis_pump = 0
weight_scale2_ng = 0
leak = 0
particle_counter = 0
s_aas_mas_tcp = 0
s_fipa = 0
leader_follower = 0


for items in element_list.index:

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:

        #Drive
        case ['M', 'SC', 'TSA+', 'V']:
            #Drive
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_DRIVE']
            drive_number = drive_number + 1 
            element_list.at[items, 'SW_Module_Nr'] = drive_number

            #Motor Conn 1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            motor_number = motor_number + 1
            element_list.at[items, 'Connection1_Nr'] = motor_number

            #Reg Conn 2
            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection2_Nr'] = reg_number

            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #value_of_Lim_hi_lo = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
            #element_list.at[items, 'Connection3_Nr'] = (element_list.at[1.840, 'SW_Module_Nr'])
            
            #get value in one line of code:
            element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']

            #Other way:
            #element_list.Connection3_Nr[items] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']


        #Motor Simple
        case ['M', 'SC', 'V', 'XA'] | ['V', 'YC'] | ['GIA', 'K', 'Y', 'YC'] | ['GIA', 'V', 'Y', 'YC'] | ['FIA+', 'H', 'YC']: 
            #Motor + #Reg
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            motor_number = motor_number + 1
            element_list.at[items, 'SW_Module_Nr'] = motor_number

        #Motor complex but no drive
        case ['H', 'M', 'NC'] | ['EC', 'TSA+']:
            #Motor + #Reg
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION["S_MOTOR1"]
            motor_number = motor_number + 1
            element_list.at[items, 'SW_Module_Nr'] = motor_number

            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection1_Nr'] = reg_number

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            element_list.at[items, 'Connection2_Nr'] = -1

        #Limit hi lo
        case ['PDI'] | ['PDIS'] | ['TIC'] | ['FICA±'] | ['TIA±'] | ['TICA±'] | ['MIA±'] | ['SIA±'] | ['M', 'P', 'QIA+', 'XA'] | ['QIA-'] | ['QSA+'] | ['PDICA±'] | ['PSA-', 'V']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        #elmt_on_off
        case ['M', 'P'] | ['L']:  
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_ELMT_ON_OFF']
            elmt_on_off = elmt_on_off + 1
            element_list.at[items, 'SW_Module_Nr'] = elmt_on_off

        #valve_flap
        case ['GSA+', 'GSA-', 'V', 'Y'] | ['GSA+', 'GSA-', 'K', 'Y'] | ['V']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_VALVE_FLAP']
            valve_flap = valve_flap + 1
            element_list.at[items, 'SW_Module_Nr'] = valve_flap

        #limit_switch
        case ['FSA-'] | ['GZA-'] | ['GSA+'] | ['GSA-'] | ['LSA-'] : 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_SWITCH']
            limit_switch = limit_switch + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_switch

        #WICA
        case ['WICA±']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_WEIGHT_SCALE2_NG']
            weight_scale2_ng = weight_scale2_ng + 1
            element_list.at[items, 'SW_Module_Nr'] = weight_scale2_ng

            #Limit hi lo conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = 'TODO'

        #Servo mousehole
        case ['GIA+', 'GIA-', 'M', 'SC', 'XA']:
            #Limit hi lo conn1
            element_list.at[items, 'SW_Module'] = "S_SERVO_MOUSEHOLE"
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = 'TODO'
        
        #Particle counter
        case ['FA-', 'QIA+', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_PMS_MODBUS_TCP']
            particle_counter = particle_counter + 1
            element_list.at[items, 'SW_Module_Nr'] = particle_counter

            

        #Air Sampler MAS TCP
        case ['P', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_AAS_MAS_TCP']
            s_aas_mas_tcp = s_aas_mas_tcp + 1
            element_list.at[items, 'SW_Module_Nr'] = s_aas_mas_tcp

        #Airdryer
        case ['T', 'XA']:
            element_list.at[items, 'SW_Module'] = 'Air dryer Munter or similar'
            element_list.at[items, 'SW_Module_Nr'] = 1

        #FiPa
        case ['M', 'XA']:
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_FIPA']
            s_fipa = s_fipa + 1
            element_list.at[items, 'SW_Module_Nr'] = s_fipa

        #SIS pumps: Controlled though SIS module 
        case ['M', 'P', 'SC']:
            element_list.at[items, 'SW_Module'] = '* SIS Pump *'
            sis_pump = sis_pump + 1
            element_list.at[items, 'SW_Module_Nr'] = sis_pump


        ## Artifcial sensors
        case 'LEAK':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LEAK']
            leak = leak + 1
            element_list.at[items, 'SW_Module_Nr'] = leak

            #Limit hi lo conn1
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']

        case '[702] [17°C < 28°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo
        
        case '[118] [17°C < 28°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[114] [15°C < 21°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[708] [30°C < 40°C]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[114] [< 100%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[702] [15%RH < 20%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[118] [< 100%RH]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[114] [> 0.45m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[118] [> 0.20m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[709] [> 0.45m/s]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[114] [>30Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[118] [>15Pa]': 
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo
        
        case '[108] [60Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case '[105] [<-200Pa >200Pa]':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'SW_Module_Nr'] = limit_hi_lo

        case 'AHU01':
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            reg_number = reg_number + 1
            element_list.at[items, 'SW_Module_Nr'] = reg_number
            
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION['S_LIMIT_HI_LO']
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection1_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.800].iloc[0]['SW_Module_Nr']

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #reg_number = reg_number + 1
            element_list.at[items, 'Connection2_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.655].iloc[0]['Connection1_Nr']

            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION['S_REG_CONT']
            #reg_number = reg_number + 1
            element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == 1.680].iloc[0]['Connection1_Nr']

        #case _:
            #print("Yikers. I don't know that one.")


In [50]:
print(element_list.to_string())

    Prop_PID_Number         Prop_PID_Element                 Tag_Number_Definition                    SW_Module SW_Module_Nr    Connection1 Connection1_Nr    Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
0             1.100           [M, SC, V, XA]                        Intake air fan                     S_MOTOR1            1            NaN            NaN            NaN            NaN            NaN            NaN         NaN            NaN
1             1.108                    [PDI]           Diferential pressure sensor                S_LIMIT_HI_LO            1            NaN            NaN            NaN            NaN            NaN            NaN         NaN            NaN
2             1.150         [M, SC, TSA+, V]                       Exhaust air fan                      S_DRIVE            1       S_MOTOR1              2     S_REG_CONT              1  S_LIMIT_HI_LO            NaN         NaN            NaN
3             1.158             

In [51]:
#Works if PID is the index
#value =(element_list.at[1.840, 'SW_Module_Nr'])
#print(value)

In [52]:
#Update with the S_limit hi lo value
for items in element_list.index:
    
    #print(element_list.Prop_PID_Number[items])

    response_code = element_list.at[items, 'Prop_PID_Element'] 

    match response_code:
        #Drive
        case ['M', 'SC', 'TSA+', 'V']:
            #Limit hi lo conn3
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]

            #If element is 1.150, get info from 1.840, 2.150 get info from 2.840
            y840 = str("%.3f" % element_list.Prop_PID_Number[items])
            y840 = float(y840[0] + '.840')
            element_list.Connection3_Nr[items] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SW_Module_Nr']
            #or
            #element_list.at[items, 'Connection3_Nr'] = element_list[element_list['Prop_PID_Number'] == y840].iloc[0]['SW_Module_Nr']

            #Y.840 #only when PID is index
            #element_list.at[items, 'Connection3_Nr'] = (element_list.at[y840, 'SW_Module_Nr']) 


        #SIS Pumps - Either ask user or GUI selection for every sis module
        #case ['M', 'P', 'SC']:
            
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.800, 'SW_Module_Nr']) #Y.800 (temp) - Ask user

            #element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #element_list.at[items, 'Connection3_Nr'] = (element_list.at[2.810, 'SW_Module_Nr']) #Y.800 (humidity) - Ask user
            


        #1.655 [H, M, NC] Control valve (Cool water control)
        case ['H', 'M', 'NC']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Connection2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SW_Module_Nr']
            

        #1.680 [D, EC, TSA+]  Electric heating
        case ['EC', 'TSA+']: 
            #element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            y688 = str("%.3f" % element_list.Prop_PID_Number[items])
            y688 = float(y688[0] + '.688')
            element_list.Connection2_Nr[items] = element_list[element_list['Prop_PID_Number'] == y688].iloc[0]['SW_Module_Nr']

"""
         #Leader Follower
        case ['W']: #1.650 heat exchanger
            element_list.at[items, 'SW_Module'] = SOFTWARE_MODE_DEFINITION['S_LEADER_FOLLOWER']
            leader_follower = leader_follower + 1
            element_list.at[items, 'SW_Module_Nr'] = leader_follower

            #Reg
            element_list.at[items, 'Connection1'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            reg_number = reg_number + 1
            element_list.at[items, 'Connection1_Nr'] = reg_number

            element_list.at[items, 'Connection2'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]
            #limit_hi_lo = limit_hi_lo + 1
            element_list.at[items, 'Connection2_Nr'] = (element_list.at[1.800, 'SW_Module_Nr'])#Temperature working chamber

            #Reg Conn 2
            element_list.at[items, 'Connection3'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            element_list.at[items, 'Connection3_Nr'] = (element_list.at[1.655, 'Connection1_Nr']) #Reg number of 1.655

            #Reg Conn 3
            element_list.at[items, 'Connection4'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]
            element_list.at[items, 'Connection4_Nr'] = (element_list.at[1.680, 'Connection1_Nr']) #Reg number of 1.680

"""

        


'\n         #Leader Follower\n        case [\'W\']: #1.650 heat exchanger\n            element_list.at[items, \'SW_Module\'] = SOFTWARE_MODE_DEFINITION[\'S_LEADER_FOLLOWER\']\n            leader_follower = leader_follower + 1\n            element_list.at[items, \'SW_Module_Nr\'] = leader_follower\n\n            #Reg\n            element_list.at[items, \'Connection1\'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]\n            reg_number = reg_number + 1\n            element_list.at[items, \'Connection1_Nr\'] = reg_number\n\n            element_list.at[items, \'Connection2\'] = SOFTWARE_MODE_DEFINITION["S_LIMIT_HI_LO"]\n            #limit_hi_lo = limit_hi_lo + 1\n            element_list.at[items, \'Connection2_Nr\'] = (element_list.at[1.800, \'SW_Module_Nr\'])#Temperature working chamber\n\n            #Reg Conn 2\n            element_list.at[items, \'Connection3\'] = SOFTWARE_MODE_DEFINITION["S_REG_CONT"]\n            element_list.at[items, \'Connection3_Nr\'] = (element_list.at[1.655, \'C

In [53]:
#


In [54]:
print(element_list.to_string())

    Prop_PID_Number         Prop_PID_Element                 Tag_Number_Definition                    SW_Module SW_Module_Nr    Connection1 Connection1_Nr    Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
0             1.100           [M, SC, V, XA]                        Intake air fan                     S_MOTOR1            1            NaN            NaN            NaN            NaN            NaN            NaN         NaN            NaN
1             1.108                    [PDI]           Diferential pressure sensor                S_LIMIT_HI_LO            1            NaN            NaN            NaN            NaN            NaN            NaN         NaN            NaN
2             1.150         [M, SC, TSA+, V]                       Exhaust air fan                      S_DRIVE            1       S_MOTOR1              2     S_REG_CONT              1  S_LIMIT_HI_LO             21         NaN            NaN
3             1.158             

In [55]:
#Export to excel
"""
# create the writer object
writer = pd.ExcelWriter('Element_list_export.xlsx')
# write dataframe to excel
element_list.to_excel(writer)
# save the excel
writer.save()
# close excel
writer.close()
"""

"\n# create the writer object\nwriter = pd.ExcelWriter('Element_list_export.xlsx')\n# write dataframe to excel\nelement_list.to_excel(writer)\n# save the excel\nwriter.save()\n# close excel\nwriter.close()\n"

Init group element list

In [56]:
#Create a new dataframe, group elements by binaries paar, later group by SW_modules, SIS, Gasket, etc...
group_element_list = pd.DataFrame()
#group_element_list = pd.DataFrame(columns=['Prop_PID_Number','Tag_Number_Definition','SW_Module','SW_Module_Nr','Connection1','Connection1_Nr'])
print(group_element_list)

Empty DataFrame
Columns: []
Index: []


#element_list.index[element_list.Prop_PID_Number == 1.108].to_flat_index()

#Return the value of element
#response = element_list[element_list['Prop_PID_Number'] == 1.840].iloc[0]['SW_Module_Nr']
#print(response)

#Return index position of an element given the pid number (return list, use the first element of the list ([0]))
#index_of = element_list.index[element_list['Prop_PID_Number'] == 1.840].to_list()
#print(index_of[0]) #[0] first element, [-1] last element

#element_list.SW_Module_Nr[index_of[0]] = 77

#Write information in a cell:
#indice = 3
#using name
#element_list.Prop_PID_Number[indice] = 'Change pid number'
#element_list.Connection3_Nr[indice] = 11
#using index
#element_list.iloc[2,10]=99 


#element_list.iloc[75,0]

In [57]:
#lenght of a list
index_of = element_list.index[element_list['Prop_PID_Number'] == 1.150].to_list()

print(len(index_of))
print(index_of)

#element_list.loc[index_of].count(axis=1).max()

element_list.notnull()


1
[2]


,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,True,True,True,True,True,True,True,True,True,True,True,True,True
155,True,True,True,True,True,True,True,True,True,True,True,True,True
156,True,True,True,True,True,True,True,True,True,True,True,True,True
157,True,True,True,True,True,True,True,True,True,True,True,True,True


In [58]:


#143           AHU01                    AHU01                Leader/Follower AHU 01                   S_REG_CONT            4  S_LIMIT_HI_LO              ?     S_REG_CONT              ?     S_REG_CONT                                          



#AHU01
#Drive with all 12 #4 * (4-1)
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with slimit

#Moto with all
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with slimit

#Reg with all
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with slimit

#Limit with all
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with slimit


#loc[2] == 1.150
#Drive with all 12 #4 * (4-1)
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with slimit

#Moto with all
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with slimit

#Reg with all
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with slimit

#Limit with all
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[2], ignore_index=True) #with slimit

#loc[4] == 1.160

#Simple motor
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[5], ignore_index=True) #without reg/limit


#3 * (3-1) = 6
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[43], ignore_index=True) #without reg/limit

group_element_list = group_element_list.append(element_list.loc[35], ignore_index=True) #without reg/limit

#3 * (3-1) = 6
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit
group_element_list = group_element_list.append(element_list.loc[45], ignore_index=True) #without reg/limit

group_element_list = group_element_list.append(element_list.loc[51], ignore_index=True) #without reg/limit


"""
#Motor with REG
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #with sreg
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #with slimit
#Reg with all
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #with slimit
#Limit with all
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #With motor
group_element_list = group_element_list.append(element_list.loc[4], ignore_index=True) #with sreg
"""

group_element_list

C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\3478914864.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #With motor
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\3478914864.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with sreg
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\3478914864.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  group_element_list = group_element_list.append(element_list.loc[143], ignore_index=True) #with slimit
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\3478914864.py:12: FutureWarning: 

,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
1,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
2,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
3,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
4,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
5,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
6,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
7,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
8,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
9,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN


How to Get Cell Value from Pandas DataFrame?

dataframe[‘column_name’].loc[dataframe.index[row_number]]

In [59]:
#list of index where 1.150 is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 1.150].to_list()

print(index_of)
#Drive with all

#Drive witn motor - in first element [index_of[0] - from first element, copy connection1 to connection1
#group_element_list.Connection1[index_of[0]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
#group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

group_element_list.Connection1[index_of[2]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3

#Motor with all - Copy motor to first position first
#3 motors
group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[4]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[3]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[3]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[5]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


#reg with all 

#3 reg
group_element_list.SW_Module[index_of[6]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[6]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[7]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[7]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[8]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[8]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[6]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[6]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[7]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[7]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[8]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[8]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


#limit with all 

#3 reg
group_element_list.SW_Module[index_of[9]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[9]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[10]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[10]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[11]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[11]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[9]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[9]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[10]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[10]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[11]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[11]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2454275439.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2454275439.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2454275439.py:14: SettingWithCopyWarning: 
A valu

In [60]:
#list of index where 1.150 is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 'AHU01'].to_list()

print(index_of)
#Drive with all

#Drive witn motor - in first element [index_of[0] - from first element, copy connection1 to connection1
#group_element_list.Connection1[index_of[0]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
#group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

group_element_list.Connection1[index_of[2]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3

#Motor with all - Copy motor to first position first
#3 motors
group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[4]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[3]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[3]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[5]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


#reg with all 

#3 reg
group_element_list.SW_Module[index_of[6]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[6]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[7]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[7]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[8]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[8]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[6]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[6]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[7]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[7]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[8]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[8]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


#limit with all 

#3 reg
group_element_list.SW_Module[index_of[9]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[9]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[10]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[10]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[11]] = group_element_list['Connection3'].loc[group_element_list.index[index_of[0]]] #Motor - Reg 
group_element_list.SW_Module_Nr[index_of[11]] = group_element_list['Connection3_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Reg number

#3motors with others (drive, reg, limit)
group_element_list.Connection1[index_of[9]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[9]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[10]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[10]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number
group_element_list.Connection1[index_of[11]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor - Drive
group_element_list.Connection1_Nr[index_of[11]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor - Drive number


[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]


C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2958492424.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2958492424.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2958492424.py:14: SettingWithCopyWarning: 
A valu

In [61]:


#Motor if we had S_REG for them
#####################################################################################################################################################

#list of index where 1.160 is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 1.655].to_list()

print(index_of)
#Drive with all

#Drive witn motor - in first element [index_of[0] - from first element, copy connection1 to connection1
#group_element_list.Connection1[index_of[0]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
#group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

#SW_Moduel: MOTOR , CONNECTION 1: REG , CONNECTION 2: LIMIT

#MOTOR : REG
#MOTOR : LIMIT
#group_element_list.Connection1[index_of[1]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #MOTOR : REG
#group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : REG
group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT


#group_element_list.Connection1[index_of[2]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
#group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3

#REG : 
#REG : 
group_element_list.SW_Module[index_of[2]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[2]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

#LIMIT:
#LIMIT: 
group_element_list.SW_Module[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[5]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number


#REG: MOTOR
#REG: LIMIT
group_element_list.Connection1[index_of[2]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.Connection1_Nr[index_of[2]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.Connection1[index_of[3]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #LIMIT
group_element_list.Connection1_Nr[index_of[3]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #LIMIT number

#LIMIT: MOTOR
#LIMIT: REG
group_element_list.Connection1[index_of[4]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.Connection1_Nr[index_of[4]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.Connection1[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #reg 
group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #reg number





[26, 27, 28, 29, 30, 31]


C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2772599907.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2772599907.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\2772599907.py:29: SettingWithCopyWarning: 
A value is trying

In [62]:
#Motor if we had S_REG for them
#####################################################################################################################################################

#list of index where 1.160 is:
index_of = group_element_list.index[group_element_list['Prop_PID_Number'] == 1.680].to_list()

print(index_of)

#Drive with all

#Drive witn motor - in first element [index_of[0] - from first element, copy connection1 to connection1
#group_element_list.Connection1[index_of[0]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #REG - Connection2
#group_element_list.Connection1_Nr[index_of[0]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #REG - Connection2-Nr

#SW_Moduel: MOTOR , CONNECTION 1: REG , CONNECTION 2: LIMIT

#MOTOR : REG
#MOTOR : LIMIT
#group_element_list.Connection1[index_of[1]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #MOTOR : REG
#group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : REG
group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT


#group_element_list.Connection1[index_of[2]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3
#group_element_list.Connection1_Nr[index_of[2]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #HILO - Connection3

#REG : 
#REG : 
group_element_list.SW_Module[index_of[2]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[2]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[3]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[3]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

#LIMIT:
#LIMIT: 
group_element_list.SW_Module[index_of[4]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.SW_Module_Nr[index_of[4]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.SW_Module[index_of[5]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #Motor 
group_element_list.SW_Module_Nr[index_of[5]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number

#REG: MOTOR
#REG: LIMIT
group_element_list.Connection1[index_of[2]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.Connection1_Nr[index_of[2]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.Connection1[index_of[3]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #LIMIT
group_element_list.Connection1_Nr[index_of[3]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #LIMIT number

#LIMIT: MOTOR
#LIMIT: REG
group_element_list.Connection1[index_of[4]] = group_element_list['SW_Module'].loc[group_element_list.index[index_of[0]]] #Motor
group_element_list.Connection1_Nr[index_of[4]] = group_element_list['SW_Module_Nr'].loc[group_element_list.index[index_of[0]]] #Motor number
group_element_list.Connection1[index_of[5]] = group_element_list['Connection1'].loc[group_element_list.index[index_of[0]]] #reg 
group_element_list.Connection1_Nr[index_of[5]] = group_element_list['Connection1_Nr'].loc[group_element_list.index[index_of[0]]] #reg number

[33, 34, 35, 36, 37, 38]


C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\1631276612.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1[index_of[1]] = group_element_list['Connection2'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\1631276612.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group_element_list.Connection1_Nr[index_of[1]] = group_element_list['Connection2_Nr'].loc[group_element_list.index[index_of[0]]] #MOTOR : LIMIT
C:\Users\marten1\AppData\Local\Temp\ipykernel_14816\1631276612.py:30: SettingWithCopyWarning: 
A value is trying

In [63]:
#Copy s_reg
#print(group_element_list['Connection2'].loc[group_element_list.index[index_of[2]]])



In [64]:
#Motor with all

In [65]:
group_element_list

,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
0,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
1,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_REG_CONT,2,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
2,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,4,S_REG_CONT,3,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
3,AHU01,AHU01,Leader/Follower AHU 01,S_LIMIT_HI_LO,12,S_REG_CONT,4,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
4,AHU01,AHU01,Leader/Follower AHU 01,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
5,AHU01,AHU01,Leader/Follower AHU 01,S_LIMIT_HI_LO,12,S_REG_CONT,3,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
6,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,2,S_REG_CONT,4,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
7,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,2,S_LIMIT_HI_LO,12,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
8,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,2,S_REG_CONT,3,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN
9,AHU01,AHU01,Leader/Follower AHU 01,S_REG_CONT,3,S_REG_CONT,4,S_REG_CONT,2,S_REG_CONT,3,NaN,NaN


In [66]:
#Delete unneccessary columns
group_element_list = group_element_list.drop(['Prop_PID_Element','Connection2','Connection2_Nr','Connection3','Connection3_Nr','Connection4','Connection4_Nr'], axis=1)

In [67]:
gdrive = group_element_list.groupby(['SW_Module','SW_Module_Nr','Prop_PID_Number','Tag_Number_Definition','Connection1','Connection1_Nr'])
gdrive.first()

Empty DataFrame
Columns: []
Index: [(S_DRIVE, 1, 1.15, Exhaust air fan, S_LIMIT_HI_LO, 21), (S_DRIVE, 1, 1.15, Exhaust air fan, S_MOTOR1, 2), (S_DRIVE, 1, 1.15, Exhaust air fan, S_REG_CONT, 1), (S_LIMIT_HI_LO, 8, 1.655, Control valve, S_MOTOR1, 10), (S_LIMIT_HI_LO, 8, 1.655, Control valve, S_REG_CONT, 2), (S_LIMIT_HI_LO, 8, 1.68, Electric heating, S_MOTOR1, 11), (S_LIMIT_HI_LO, 8, 1.68, Electric heating, S_REG_CONT, 3), (S_LIMIT_HI_LO, 12, AHU01, Leader/Follower AHU 01, S_REG_CONT, 2), (S_LIMIT_HI_LO, 12, AHU01, Leader/Follower AHU 01, S_REG_CONT, 3), (S_LIMIT_HI_LO, 12, AHU01, Leader/Follower AHU 01, S_REG_CONT, 4), (S_LIMIT_HI_LO, 21, 1.15, Exhaust air fan, S_DRIVE, 1), (S_LIMIT_HI_LO, 21, 1.15, Exhaust air fan, S_MOTOR1, 2), (S_LIMIT_HI_LO, 21, 1.15, Exhaust air fan, S_REG_CONT, 1), (S_MOTOR1, 2, 1.15, Exhaust air fan, S_DRIVE, 1), (S_MOTOR1, 2, 1.15, Exhaust air fan, S_LIMIT_HI_LO, 21), (S_MOTOR1, 2, 1.15, Exhaust air fan, S_REG_CONT, 1), (S_MOTOR1, 3, 1.16, Exhaust air fan, NaN, NaN), (S_MOTOR1, 4, 1.2, Recirculation fan, NaN, NaN), (S_MOTOR1, 7, 1.55, Exhaust air flap, NaN, NaN), (S_MOTOR1, 10, 1.655, Control valve, S_LIMIT_HI_LO, 8), (S_MOTOR1, 10, 1.655, Control valve, S_REG_CONT, 2), (S_MOTOR1, 11, 1.68, Electric heating, S_LIMIT_HI_LO, 8), (S_MOTOR1, 11, 1.68, Electric heating, S_REG_CONT, 3), (S_MOTOR1, 12, 1.707, Needle valve compressed air, NaN, NaN), (S_REG_CONT, 1, 1.15, Exhaust air fan, S_DRIVE, 1), (S_REG_CONT, 1, 1.15, Exhaust air fan, S_LIMIT_HI_LO, 21), (S_REG_CONT, 1, 1.15, Exhaust air fan, S_MOTOR1, 2), (S_REG_CONT, 2, 1.655, Control valve, S_LIMIT_HI_LO, 8), (S_REG_CONT, 2, 1.655, Control valve, S_MOTOR1, 10), (S_REG_CONT, 2, AHU01, Leader/Follower AHU 01, S_LIMIT_HI_LO, 12), (S_REG_CONT, 2, AHU01, Leader/Follower AHU 01, S_REG_CONT, 3), (S_REG_CONT, 2, AHU01, Leader/Follower AHU 01, S_REG_CONT, 4), (S_REG_CONT, 3, 1.68, Electric heating, S_LIMIT_HI_LO, 8), (S_REG_CONT, 3, 1.68, Electric heating, S_MOTOR1, 11), (S_REG_CONT, 3, AHU01, Leader/Follower AHU 01, S_LIMIT_HI_LO, 12), (S_REG_CONT, 3, AHU01, Leader/Follower AHU 01, S_REG_CONT, 2), (S_REG_CONT, 3, AHU01, Leader/Follower AHU 01, S_REG_CONT, 4), (S_REG_CONT, 4, AHU01, Leader/Follower AHU 01, S_LIMIT_HI_LO, 12), (S_REG_CONT, 4, AHU01, Leader/Follower AHU 01, S_REG_CONT, 2), (S_REG_CONT, 4, AHU01, Leader/Follower AHU 01, S_REG_CONT, 3)]

In [68]:
gdrive.first().to_excel('GroupElementList.xlsx')

End _Group element list

In [69]:
element_list.count()

Prop_PID_Number          159
Prop_PID_Element         159
Tag_Number_Definition    158
SW_Module                159
SW_Module_Nr             159
Connection1              159
Connection1_Nr           159
Connection2              159
Connection2_Nr           159
Connection3              159
Connection3_Nr           159
Connection4              159
Connection4_Nr           159
dtype: int64

In [70]:
#Drives
element_list_drives = element_list[element_list['SW_Module'] == 'S_DRIVE']
print(element_list_drives.to_string())

  Prop_PID_Number  Prop_PID_Element Tag_Number_Definition SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr    Connection3 Connection3_Nr Connection4 Connection4_Nr
2           1.150  [M, SC, TSA+, V]       Exhaust air fan   S_DRIVE            1    S_MOTOR1              2  S_REG_CONT              1  S_LIMIT_HI_LO             21         NaN            NaN


In [71]:
#Motor
element_list_motors = element_list[element_list['SW_Module'] == 'S_MOTOR1']
print(element_list_motors.to_string())

    Prop_PID_Number Prop_PID_Element        Tag_Number_Definition SW_Module SW_Module_Nr Connection1 Connection1_Nr    Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
0             1.100   [M, SC, V, XA]               Intake air fan  S_MOTOR1            1         NaN            NaN            NaN            NaN         NaN            NaN         NaN            NaN
4             1.160   [M, SC, V, XA]              Exhaust air fan  S_MOTOR1            3         NaN            NaN            NaN            NaN         NaN            NaN         NaN            NaN
5             1.200   [M, SC, V, XA]            Recirculation fan  S_MOTOR1            4         NaN            NaN            NaN            NaN         NaN            NaN         NaN            NaN
7             1.210   [M, SC, V, XA]            Recirculation fan  S_MOTOR1            5         NaN            NaN            NaN            NaN         NaN            NaN         NaN            NaN


In [72]:
gm = element_list_motors.groupby('SW_Module')
gm.first()


,Prop_PID_Number,Prop_PID_Element,Tag_Number_Definition,SW_Module_Nr,Connection1,Connection1_Nr,Connection2,Connection2_Nr,Connection3,Connection3_Nr,Connection4,Connection4_Nr
SW_Module,,,,,,,,,,,,
S_MOTOR1,1.100,"[M, SC, V, XA]",Intake air fan,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
#Valve
element_list_valve_flap = element_list[element_list['SW_Module'] == 'S_VALVE_FLAP']
print(element_list_valve_flap.to_string())

    Prop_PID_Number    Prop_PID_Element                 Tag_Number_Definition     SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
13            1.305  [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
17            1.315  [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
21            1.325  [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
25            1.335  [GSA+, GSA-, V, Y]                          3-way valve   S_VALVE_FLAP            4         NaN            NaN         NaN     

In [74]:
#Element on off
element_list_elmt_on_off = element_list[element_list['SW_Module'] == 'S_ELMT_ON_OFF']
print(element_list_elmt_on_off.to_string())

    Prop_PID_Number Prop_PID_Element Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
12            1.304           [M, P]          Vacuum pump   S_ELMT_ON_OFF            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
16            1.314           [M, P]          Vacuum pump   S_ELMT_ON_OFF            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
20            1.324           [M, P]          Vacuum pump   S_ELMT_ON_OFF            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
24            1.334           [M, P]          Vacuum pump   S_ELMT_ON_OFF            4         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
41            1.651      

In [75]:
element_list_limit_switch = element_list[element_list['SW_Module'] == 'S_LIMIT_SWITCH']
print(element_list_limit_switch.to_string())

    Prop_PID_Number Prop_PID_Element Tag_Number_Definition       SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
14            1.308           [FSA-]           Flow-Switch  S_LIMIT_SWITCH            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
18            1.318           [FSA-]           Flow-Switch  S_LIMIT_SWITCH            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
22            1.328           [FSA-]           Flow-Switch  S_LIMIT_SWITCH            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
26            1.338           [FSA-]           Flow-Switch  S_LIMIT_SWITCH            4         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
49            1.705 

In [76]:
element_list_limit_hi_lo = element_list[element_list['SW_Module'] == 'S_LIMIT_HI_LO']
print(element_list_limit_hi_lo.to_string())

    Prop_PID_Number        Prop_PID_Element          Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
1             1.108                   [PDI]    Diferential pressure sensor  S_LIMIT_HI_LO            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
3             1.158                   [PDI]    Diferential pressure sensor  S_LIMIT_HI_LO            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
6             1.208                   [PDI]   Diferential pressure sensor   S_LIMIT_HI_LO            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
8             1.218                   [PDI]    Diferential pressure sensor  S_LIMIT_HI_LO            4         NaN            NaN         NaN            NaN

In [77]:
element_list_leak = element_list[element_list['SW_Module'] == 'S_LEAK']
print(element_list_leak.to_string())

    Prop_PID_Number Prop_PID_Element Tag_Number_Definition SW_Module SW_Module_Nr    Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
154      1.825_leak             LEAK      Flow rate sensor    S_LEAK            1  S_LIMIT_HI_LO             21         NaN            NaN         NaN            NaN         NaN            NaN


In [78]:
#Particle counter modbus tcp
element_list_pms_modbus_tcp = element_list[element_list['SW_Module'] == 'S_PMS_MODBUS_TCP']
print(element_list_pms_modbus_tcp.to_string())


    Prop_PID_Number Prop_PID_Element Tag_Number_Definition         SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
11            1.300  [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
15            1.310  [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
19            1.320  [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
23            1.330  [FA-, QIA+, XA]      Particle counter  S_PMS_MODBUS_TCP            4         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
124       

In [79]:
#Air sampler modbus
element_list_aas_mas_tcp = element_list[element_list['SW_Module'] == 'S_AAS_MAS_TCP']
print(element_list_aas_mas_tcp.to_string())

    Prop_PID_Number Prop_PID_Element Tag_Number_Definition      SW_Module SW_Module_Nr Connection1 Connection1_Nr Connection2 Connection2_Nr Connection3 Connection3_Nr Connection4 Connection4_Nr
27            1.350          [P, XA]           Air sampler  S_AAS_MAS_TCP            1         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
28            1.360          [P, XA]           Air sampler  S_AAS_MAS_TCP            2         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
29            1.370          [P, XA]           Air sampler  S_AAS_MAS_TCP            3         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN
127           2.350          [P, XA]           Air sampler  S_AAS_MAS_TCP            4         NaN            NaN         NaN            NaN         NaN            NaN         NaN            NaN


In [80]:
#element_list_motors = element_list[element_list['SW_Module'] == 'S_MOTOR1']
#print(element_list_motors.to_string())

In [81]:
#Group by SW_Module
#el_sortby_sw_module = element_list.groupby(['SW_Module']).agg(list).copy()
#print(el_sortby_sw_module.to_string())

In [82]:
#Group by pdi number again
#this time
#Group by PID number
#element_list = element_list.groupby(['Prop_PID_Number']).agg(list).copy()

In [83]:
#print(element_list.to_string())

In [84]:
#list_1100 = element_list.loc[1.100].values.tolist()
#list_1100[0].sort()

In [85]:
#print(list_1100[0])

end_new code

In [86]:
#Remove duplicates - WARNING: It removes duplicates alarms as 1.310 Particle counter 0.5 um and 5.0um (maybe add exception when is an PCounter and duplicates elements)

#df_data_excel_uniques = df_data_excel.drop_duplicates()

In [87]:
#df_data_excel_uniques.count()

In [88]:
#gkk = df_data_excel_uniques.groupby(['Prop_PID_Number', 'Prop_PID_Element'])
#gkk.first()

In [89]:
#Get uniques values from the PID Elements (the columns for the matrix)
#df_unique_elememts = df_data_excel.Prop_PID_Element.unique()
#df_unique_elememts.sort()
#print(df_unique_elememts)




#Export Unique elements to excel file - Once


In [90]:
#Create a dataframe containing the elements
#pd_automation_code = pd.DataFrame(df_unique_elememts)
#Create the writer object
#automation_code = pd.ExcelWriter('AutomationCode.xlsx')
#Write dataframe to excel file
#pd_automation_code.to_excel(automation_code)
#Save excel 
#automation_code.save()
#Close excel
#automation_code.close()


In [91]:
#Sort by PID number
#df_data_excel_sorted = df_data_excel_uniques.sort_values(by='Prop_PID_Number')

In [92]:
#Set PID number as multi-index (group by does better)
#df_data_excel_sorted.set_index(['Prop_PID_Number','Prop_PID_Element'])

In [93]:
#df_data_excel_sorted.head(1000)

In [94]:
#Add column, load data from Automation Code and mix
#df_data_excel_sorted['Automation Code'] = ''


In [95]:
#df_data_excel_sorted

In [96]:
#Load Automation Code excel file
#df_automation_code = pd.read_excel('AutomationCode.xlsx')
#df_automation_code.head()

In [97]:
#Lookup
#df = df_data_excel
#df['Automation_Code'] = ''
#print (df)

In [98]:
# Deprecated df['Automation Code'] = df.set_index('Prop_PID_Element').lookup(df_automation_code.Prop_PID_Element, df_automation_code.Automation_Code)

In [99]:
#pd.melt(df,)
#print(df.combine_first(df_automation_code))

In [100]:
#documents = [['SW_MODULE', 'SW_MODULE_NR'], ['CONN1', 'CONN1_NR'], ['CONN2', 'CONN2_NR'], ['CONN3', 'CONN3_NR'], 'AM', 'HW']
#documents = ['EL', 'AM', 'HW', 'FN']

In [101]:
#
#df[df_unique_elememts] = pd.NaT
#documents = ['EL', 'AM', 'HW', 'FN']
#df[documents] = pd.NaT
#print(df.to_string())

ALARM MATRIX

In [102]:
df_am = pd.read_excel('AM.xlsm')

df_am


,Unnamed: 0,Unnamed: 1
0,MSRDok,NaN
1,NaN,NaN
2,AM - Alarm-Matrix EN,NaN
3,NaN,NaN
4,Customer,NaN
5,User,NaN
6,Identification,NaN
7,NaN,NaN
8,Project name,Vorlagen für MSR GAMP Dokumente
9,Revision date,16.02.2022


In [103]:
df_am = pd.read_excel('AM.xlsm', sheet_name="Alarm Matrix")

c:\Users\marten1\Anaconda3\envs\threeten\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [104]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#pd.set_option('display.width', 1000)
#pd.set_option('display.colheader_justify', 'center')
#pd.set_option('display.precision', 3)

#display(df_am)